In [1]:
from tool import *
from openpyxl import Workbook
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.drawing.image import Image
from openpyxl.styles import PatternFill, Alignment, Font, Border, Side

In [2]:
with open('D:/Data/Shipment Plan_Adjusted_new.DB', 'rb') as f:
    spdb = pickle.load(f)

with open('D:/Data/ODM PO.db', 'rb') as f:
    po = pickle.load(f)
    po = po[po['Cancel'] == 'N']

with open(f'D:/Data/ODM_SR.db', 'rb') as f:
    sr = pickle.load(f)

with open('D:/Data/TCL PP_result.bin', 'rb') as f:
    ppr = pickle.load(f)

with open('D:/Data/TCL PP_new.bin', 'rb') as f:
    pp_db = pickle.load(f)

In [3]:
ver_table = spdb.groupby(['Planweek', 'From Site', 'Category', 'Created_at']).sum('SP').reset_index()
ver_table = ver_table[ver_table['From Site'].str.contains('TCL')]
ver_table.iloc[-10:, ]

,Planweek,From Site,Category,Created_at,SP
878,2024-02-26(W09),OS_TCL_CN_P,Final26w,202402271452,508784.0
879,2024-02-26(W09),OS_TCL_CN_P,Final52w,202402281001,630469.0
880,2024-02-26(W09),OS_TCL_CN_P,Latest26w,202402261515,509742.0
881,2024-02-26(W09),OS_TCL_CN_P,Latest26w,202402270919,509574.0
914,2024-03-04(W10),OS_TCL_CN_P,Final26w,202403051512,467295.0
915,2024-03-04(W10),OS_TCL_CN_P,Final52w,202403060851,567861.0
916,2024-03-04(W10),OS_TCL_CN_P,Latest26w,202403040906,461485.0
917,2024-03-04(W10),OS_TCL_CN_P,Latest26w,202403041516,468575.0
918,2024-03-04(W10),OS_TCL_CN_P,Latest26w,202403050912,469114.0
919,2024-03-04(W10),OS_TCL_CN_P,Latest26w,202403070917,467295.0


In [4]:
i = 914
# TCL SP 구하기
p_wk = spdb.groupby(['Planweek', 'From Site', 'Category', 'Created_at']).sum('SP').reset_index().iloc[i, :4]['Planweek']
sp = pd.merge(spdb, spdb.groupby(['Planweek', 'From Site', 'Category', 'Created_at']).sum('SP').reset_index().iloc[[i], :4])
sp = sp.groupby(['Mapping Model.Suffix', 'Week Name']).sum('SP').reset_index()
sp = sp.rename(columns={'SP':'QTY'})
sp['Category'] = '2_SP'
sp = sp[sp['Week Name'] >= p_wk]
sr = sr.loc['OS_TCL_CN_P']
sr['Week Name'] = sr['Ship Date'].map(get_weekname)
past_sp = sr.groupby(['Mapping Model.Suffix', 'Week Name']).sum('Ship')['Ship'].reset_index()
past_sp = past_sp[past_sp['Week Name'] < p_wk]
sp = pd.concat([past_sp.rename(columns={'Ship':'QTY'}), sp]).reset_index(drop=True)
sp['Category'] = sp['Category'].bfill()

In [5]:
# 지난 주 기준의 TCL OPEN PO  구하기
sr = sr[sr['Week Name'] < p_wk] # 선적 결과에서 지난 주까지 선적된 것만 추출(이번주 이후 선적된 것은 제외함, 왜냐하면 지난 주 기준의 Open PO를 구하기 위함)
opo = pd.merge(po.loc['OS_TCL_CN_P'].reset_index(), sr.groupby('PO No.').sum('Ship')['Ship'].reset_index(), how='left').fillna(0)
opo['Week Name'] = opo['RSD'].map(get_weekname)
opo['Ship'] = opo['Ship'].astype(int)
opo['QTY'] = opo['PO'] - opo['Ship'] # PO에서 선적된 수량 차감하여 잔량 PO 수량 계산
opo = opo[opo['QTY'] > 0]
opo.loc[opo['Week Name'] < p_wk, 'Week Name'] = p_wk # Open PO 중 RSD가 지난 주까지로 된 PO는 이번주로 Carry Over된 Open PO이므로
opo = opo.groupby(['Mapping Model.Suffix', 'Week Name']).sum('QTY')['QTY'].reset_index()
opo['Category'] = '3_PO'

In [6]:
def make_pp(ver_num):
    update_ver = sorted(pp_db['Updated_at'].unique())[ver_num]
    c1 = ppr['PP Date'] < sorted(pp_db[pp_db['Updated_at'] == update_ver]['PP Date'].unique())[0]
    c2 = ppr['PP Date'] >= datetime.datetime(2024, 1, 1) # 특정월의 첫째날부터 가져오기 위한 조건
    pp = pd.concat([ppr[c1 & c2], pp_db[pp_db['Updated_at'] == update_ver]], join='inner').reset_index(drop=True)
    pp['Updated_at'] = update_ver
    return pp

In [7]:
pp1 = make_pp(-1) # 최신 PP 를 가져옴
pp2 = make_pp(-2) # 이전 Version의 PP를 가져옴
pp_sum = pd.concat([pp1, pp2]).reset_index(drop=True)
pp_sum['Month'] = pp_sum['PP Date'].apply(lambda x:f'{x.isocalendar().year}-{get_month_from_date(x):02d}')
pp_sum['Week Name'] = pp_sum['PP Date'].map(get_weekname)
pp_sum['Series'] = pp_sum['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).apply(lambda x:x.split('.')[0])
pp_sum['Day'] = pp_sum['PP Date'].dt.strftime('%a')
pp_sum['Date'] = pp_sum['PP Date'].dt.strftime('%m/%d')

In [8]:
gap_month = pp_sum.groupby(['Mapping Model.Suffix', 'Rep PMS', 'TCL BOM', 'Month', 'Week Name', 'Date', 'Day', 'Updated_at']).sum('QTY')['QTY'].unstack('Updated_at').fillna(0).astype(int)
gap_month['Gap'] = gap_month[gap_month.columns[-1]] - gap_month[gap_month.columns[0]]
gap_month = gap_month.unstack(['Month', 'Week Name']).dropna(how='all', axis=1).fillna(0).astype(int)
gap_month['Sum'] = gap_month['Gap'].sum(axis=1)
# gap_month.loc[('', '', '', '', '', 'Sum'), :] = gap_month.sum()
gap_month = gap_month.astype(int)

In [9]:
# TCL PP 상세 내역 구하기
# ppr = ppr[ppr['Updated_at'] == ppr['Updated_at'].max()] # 최신 ppr version 추출
ppr['Month'] = ppr['PP Date'].apply(lambda x:f'{x.isocalendar().year}-{get_month_from_date(x):02d}')
ppr['Week Name'] = ppr['PP Date'].map(get_weekname)
ppr['Day'] = ppr['PP Date'].dt.strftime('%a')
ppr['Date'] = ppr['PP Date'].dt.strftime('%m/%d')

tcl_pp = pp_db[pp_db['Updated_at'] == pp_db['Updated_at'].max()].reset_index(drop=True) # 최신 PP 를 가져옴
tcl_pp['Month'] = tcl_pp['PP Date'].apply(lambda x:f'{x.isocalendar().year}-{get_month_from_date(x):02d}')
tcl_pp['Day'] = tcl_pp['PP Date'].dt.strftime('%a')
tcl_pp['Date'] = tcl_pp['PP Date'].dt.strftime('%m/%d')
tcl_pp['Week Name'] = tcl_pp['PP Date'].map(get_weekname)
tcl_pp = pd.concat([ppr, tcl_pp], join='inner')
tcl_pp = tcl_pp.pivot_table('QTY', index=['Mapping Model.Suffix', 'Rep PMS', 'TCL BOM'], columns=['Month', 'Week Name', 'Date', 'Day'], aggfunc=sum).fillna(0)

In [10]:
# TCL PP 구하기(생산실적과 생산계획을 결합)
ppr = ppr.groupby(['Mapping Model.Suffix', 'Week Name']).sum('QTY').reset_index()
ppr['Category'] = '4_PP'

pp = pp_db[pp_db['Updated_at'] == pp_db['Updated_at'].max()].reset_index(drop=True) # 최신 PP 를 가져옴
pp['Week Name'] = pp['PP Date'].map(get_weekname)
pp = pp[['Mapping Model.Suffix', 'Week Name', 'QTY']]
pp = pp.groupby(['Mapping Model.Suffix', 'Week Name']).sum('QTY').reset_index()
pp['Category'] = '4_PP'
pp = pd.concat([ppr, pp]).reset_index(drop=True)

In [11]:
# PSI report 만들기
p1 = pd.concat([sp, opo, pp])[['Mapping Model.Suffix']].drop_duplicates().reset_index(drop=True) # PSI report 의 index로 사용될 전체

In [12]:
p1['Category'] = '1_BOH'

In [13]:
psi = pd.concat([p1, p1.drop('Category', axis=1)]).fillna('2_SP').reset_index(drop=True)
psi = pd.concat([psi, p1.drop('Category', axis=1)]).fillna('3_PO').reset_index(drop=True)
psi = pd.concat([psi, p1.drop('Category', axis=1)]).fillna('4_PP').reset_index(drop=True)
psi = pd.concat([psi, p1.drop('Category', axis=1)]).fillna('5_ETD').reset_index(drop=True)
psi = pd.concat([psi, p1.drop('Category', axis=1)]).fillna('6_SP Gap').reset_index(drop=True)

In [14]:
psi = psi.set_index(['Mapping Model.Suffix', 'Category'])

In [15]:
min_wk = pd.concat([sp, opo, pp])['Week Name'].min()
max_wk = pd.concat([sp, opo, pp])['Week Name'].max()
wk = min_wk
wklist = [get_weekname_from(wk, i) for i in range(100)]
wklist = wklist[:wklist.index(max_wk)+1]
psi[wklist] = 0

In [16]:
psi.update(sp.pivot_table('QTY', index=['Mapping Model.Suffix', 'Category'], columns='Week Name', aggfunc=sum).fillna(0).astype(int))
psi.update(opo.pivot_table('QTY', index=['Mapping Model.Suffix', 'Category'], columns='Week Name', aggfunc=sum).fillna(0).astype(int))
psi.update(pp.pivot_table('QTY', index=['Mapping Model.Suffix', 'Category'], columns='Week Name', aggfunc=sum).fillna(0).astype(int))

In [17]:
psi = psi.convert_dtypes().sort_index()

In [18]:
for map_model in p1['Mapping Model.Suffix']:
    for week in psi.columns:
        if week == psi.columns[0]: # 계산하는 Week가 시작하는 Week 일 경우
            current_boh = psi.loc[(map_model, '1_BOH'), week] # 금주 boh
            current_sp = psi.loc[(map_model, '2_SP'), week] # 금주 sp
            if current_boh > current_sp:
                psi.loc[(map_model, '5_ETD'), week] = current_sp
            else:
                psi.loc[(map_model, '5_ETD'), week] = current_boh
            psi.loc[(map_model, '6_SP Gap'), week] = psi.loc[(map_model, '5_ETD'), week] - current_sp
        else:
            last_boh = psi.loc[(map_model, '1_BOH'), get_weekname_from(week, -1)]
            last_pp = psi.loc[(map_model, '4_PP'), get_weekname_from(week, -1)]
            last_etd = psi.loc[(map_model, '5_ETD'), get_weekname_from(week, -1)]
            psi.loc[(map_model, '1_BOH'), week] =  last_boh + last_pp - last_etd
            current_boh = psi.loc[(map_model, '1_BOH'), week]
            current_sp = psi.loc[(map_model, '2_SP'), week]
            last_gap = psi.loc[(map_model, '6_SP Gap'), get_weekname_from(week, -1)]
            if current_boh  > current_sp - last_gap:
                psi.loc[(map_model, '5_ETD'), week] = current_sp - last_gap
            else:
                psi.loc[(map_model, '5_ETD'), week] = current_boh
            psi.loc[(map_model, '6_SP Gap'), week] = psi.loc[(map_model, '5_ETD'), week] - current_sp + last_gap

In [19]:
psi_sum = psi.stack().reset_index()
psi_sum.columns = ['Mapping Model.Suffix', 'Category', 'Week Name', 'QTY']
psi_sum.insert(0, 'Inch', psi_sum['Mapping Model.Suffix'].apply(lambda x:x[:2]))
psi_sum = psi_sum.pivot_table('QTY', index=['Inch', 'Category'], columns=['Week Name'], aggfunc=sum)
psi = psi.reset_index()

In [20]:
update_time = datetime.datetime.now().strftime('%Y%m%d%H')
filename = f'{update_time}_TCL Monitor PSI.xlsx'
with pd.ExcelWriter(f'D:/MNT/TCL/PSI/{filename}') as writer:
    psi_sum.to_excel(writer, sheet_name='PSI Summary', index=True, startrow=2)
    psi.to_excel(writer, sheet_name='TCL PSI', index=False, merge_cells=False, startrow=1)
    tcl_pp.to_excel(writer, sheet_name='TCL PP', index=True, startrow=1)
    gap_month.to_excel(writer, sheet_name='PP gap', startrow=1)

In [21]:
wb = load_workbook(f'D:/MNT/TCL/PSI/{filename}')

color = PatternFill(fgColor='9999FF', fill_type='solid') # Blue
color_boh = PatternFill(fgColor='EAE6FF', fill_type='solid') # Lavender(web)
color_sp = PatternFill(fgColor='FFEECC', fill_type='solid') # Blanched Almond
color_po = PatternFill(fgColor='E6F2FF', fill_type='solid') # Glitter
color_pp = PatternFill(fgColor='FFE6F7', fill_type='solid') # Lavender blush
color_etd = PatternFill(fgColor='E6FFEA', fill_type='solid') # Honeydew
color_sp_gap = PatternFill(fgColor='FBFFE6', fill_type='solid') # Light yellow
big_size = Font(name='맑은 고딕', size=20, bold=True, color='000099')
middle_size = Font(name='맑은 고딕', size=15, bold=True, color='000099')

border_style = Side(border_style="thin", color="000000")
border = Border(top=border_style, right=border_style, bottom=border_style, left=border_style)

In [22]:
sh = wb[wb.sheetnames[0]]
sh['A1'] = f'■ TCL Monitor PSI Summary_updated at : {update_time}'
sh['A1'].font = middle_size
sh['B2'] = 'Sub TTL'
for i in range(3, sh.max_column+1):
    col_letter = get_column_letter(i)
    sh.cell(row=2, column=i).value = f'=subtotal(9, {col_letter}4:{col_letter}{sh.max_row})'
    sh.cell(row=2, column=i).border = border
    sh.cell(row=2, column=i).number_format = '#,##0;[RED]Δ#,##0;"-"'
for cell in sh[3]:
    cell.fill = color
    cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
for row in sh.iter_rows(min_row=4, min_col=3, max_row=sh.max_row, max_col=sh.max_column):
    for cell in row:
        cell.border = border
        cell.number_format = '#,##0;[RED]Δ#,##0;"-"'
sh.freeze_panes = 'C4'
for row in sh.iter_rows(min_row=4, min_col=2):
    if row[0].value == '1_BOH':
        for cell in row:
            cell.fill = color_boh
    elif row[0].value == '2_SP':
        for cell in row:
            cell.fill = color_sp
    elif row[0].value == '3_PO':
        for cell in row:
            cell.fill = color_po
    elif row[0].value == '4_PP':
        for cell in row:
            cell.fill = color_pp
    elif row[0].value == '5_ETD':
        for cell in row:
            cell.fill = color_etd
    else:
        for cell in row:
            cell.fill = color_sp_gap

sh = wb[wb.sheetnames[1]]
sh['A1'] = f'■ TCL Monitor PSI_updated at : {update_time}'
sh['A1'].font = middle_size
sh['B1'] = 'Sub TTL'
for i in range(3, sh.max_column+1):
    col_letter = get_column_letter(i)
    sh.cell(row=1, column=i).value = f'=subtotal(9, {col_letter}3:{col_letter}{sh.max_row})'
    sh.cell(row=1, column=i).border = border
    sh.cell(row=1, column=i).number_format = '#,##0;[RED]Δ#,##0;"-"'
for cell in sh[2]:
    cell.fill = color
    cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
for row in sh.iter_rows(min_row=3, min_col=1, max_row=sh.max_row, max_col=sh.max_column):
    for cell in row:
        cell.border = border
        cell.number_format = '#,##0;[RED]Δ#,##0;"-"'
sh.freeze_panes = 'C3'
sh.column_dimensions['A'].width = 17
sh.column_dimensions['B'].width = 10

for row in sh.iter_rows(min_row=3):
    if row[1].value == '1_BOH':
        for cell in row:
            cell.fill = color_boh
    elif row[1].value == '2_SP':
        for cell in row:
            cell.fill = color_sp
    elif row[1].value == '3_PO':
        for cell in row:
            cell.fill = color_po
    elif row[1].value == '4_PP':
        for cell in row:
            cell.fill = color_pp
    elif row[1].value == '5_ETD':
        for cell in row:
            cell.fill = color_etd
    else:
        for cell in row:
            cell.fill = color_sp_gap

sh = wb[wb.sheetnames[2]]
sh['A1'] = f'■ TCL PP'
sh['A1'].font = middle_size
sh['C1'] = 'Sub TTL'

for i in range(4, sh.max_column+1):
    col_letter = get_column_letter(i)
    sh.cell(row=1, column=i).value = f'=subtotal(9, {col_letter}7:{col_letter}{sh.max_row})'
    sh.cell(row=1, column=i).border = border
    sh.cell(row=1, column=i).number_format = '#,##0;[RED]Δ#,##0;"-"'

for row in sh.iter_rows(min_row=2, min_col=len(tcl_pp.index[0]) ,max_row=2+len(tcl_pp.columns[0])-1, max_col=sh.max_column):
    for cell in row:
        cell.fill = color
        cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)

for row in sh.iter_rows(min_row=6, min_col=1 ,max_row=6, max_col=len(tcl_pp.index[0])):
    for cell in row:
        cell.fill = color
        cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)        

for row in sh.iter_rows(min_row=7, min_col=4, max_row=sh.max_row, max_col=sh.max_column):
    for cell in row:
        cell.border = border
        cell.number_format = '#,##0;[RED]Δ#,##0;"-"'
sh.freeze_panes = 'D7'
sh.column_dimensions['A'].width = 17
sh.column_dimensions['B'].width = 20
sh.column_dimensions['C'].width = 23

sh = wb[wb.sheetnames[3]]
sh['A1'] = f'■ TCL PP Gap from previous plan'
sh['A1'].font = middle_size
sh['E1'] = 'Sub TTL'

for i in range(6, sh.max_column+1):
    col_letter = get_column_letter(i)
    sh.cell(row=1, column=i).value = f'=subtotal(9, {col_letter}6:{col_letter}{sh.max_row})'
    sh.cell(row=1, column=i).border = border
    sh.cell(row=1, column=i).number_format = '#,##0;[RED]Δ#,##0;"-"'
    
for row in sh.iter_rows(min_row=2, min_col=len(gap_month.index[0]) ,max_row=2+len(gap_month.columns[0])-1, max_col=sh.max_column):
    for cell in row:
        cell.fill = color
        cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)

for row in sh.iter_rows(min_row=5, min_col=1 ,max_row=5, max_col=len(gap_month.index[0])):
    for cell in row:
        cell.fill = color
        cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
        
for row in sh.iter_rows(min_row=6, min_col=6, max_row=sh.max_row, max_col=sh.max_column):
    for cell in row:
        cell.border = border
        cell.number_format = '#,##0;[RED]Δ#,##0;"-"'
sh.freeze_panes = 'F6'
sh.column_dimensions['A'].width = 17
sh.column_dimensions['B'].width = 20
sh.column_dimensions['C'].width = 23


for name in wb.sheetnames:
    wb[name].sheet_view.showGridLines = False
wb.save(f'D:/MNT/TCL/PSI/{filename}')
wb.close()